Name -  Faria Alam

e-mail - faal00001@stud.uni-saarland.de

Matriculation number- 7011180

Name - Akhil Juneja

e-mail - akju00001@stud.uni-saarland.de

Matriculation number- 7015523

## Introduction
Semantic textual similarity deals with determining how similar a pair of text documents are. The goal of the first task is to implement a new architecture by combining the ideas from papers
- Siamese Recurrent Architectures for Learning Sentence Similarity, Jonas Mueller et. al (will be referred as the AAAI paper)
- A Structured Self-Attentive Sentence Embedding, Zhouhan Lin et. al (will be referred as the ICLR paper) <br/><br/>
Furthermore, you'd be evaluating whether the new architecture improves the results of **Siamese Recurrent Architectures for Learning Sentence Similarity, Jonas Mueller et. al**. Your overall network architecture should look similar to the following figure. 
![Untitled%20Diagram.drawio%20%281%29.png](https://raw.githubusercontent.com/shahrukhx01/ocr-test/main/download.png)
<br/><br/>


Moreover, you'd be required to implement further helper functions that these papers propose i.e., attention penalty term for loss, etc.

### SICK dataset
We will use SICK dataset throughout the project (at least in the first two tasks). To get more information about the dataset you can refer to the original [paper](http://www.lrec-conf.org/proceedings/lrec2014/pdf/363_Paper.pdf) on the dataset. You can download the dataset using one of the following links:
- [dataset page 1](https://marcobaroni.org/composes/sick.html)
- [dataset page 2](https://huggingface.co/datasets/sick)    

The relevant columns for the project are `sentence_A`, `sentence_B`, `relatedness_score`, where `relatedness_score` is the label. <br><br>
**Hint: For each task make sure to decide whether the label should be normalized or not.**<br><br>

In [7]:
!pip install datasets

     |████████████████████████████████| 312 kB 12.4 MB/s 
     |████████████████████████████████| 212 kB 49.1 MB/s 
     |████████████████████████████████| 67 kB 5.6 MB/s 
     |████████████████████████████████| 134 kB 52.6 MB/s 
     |████████████████████████████████| 1.1 MB 46.8 MB/s 
     |████████████████████████████████| 127 kB 39.3 MB/s 
     |████████████████████████████████| 271 kB 51.4 MB/s 
     |████████████████████████████████| 144 kB 51.6 MB/s 
     |████████████████████████████████| 94 kB 1.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import torch
from train import train_model
from test import evaluate_test_set
import sts_data
from importlib import reload
import numpy as np
import random
from datetime import datetime



## Part 1. Data pipeline (3 points)
Before starting working on the model, we must configure the data pipeline to load the data in the correct format. Please, implement the functions for processing the data.

### Part 1.1 Loading and preprocessing the data (1 point)
Download the SICK dataset and store it in [pandas](https://pandas.pydata.org/docs/index.html) `Dataframe`'s. You should use the official data split.  

Implement `load_data` method of `STSData` class in `sts_data.py`. The method must download the dataset and perform basic preprocessing. Minimal preprocessing required:  
1. normalize text to lower case
2. remove punctuations  
3. remove [stopwords](https://en.wikipedia.org/wiki/Stop_word) - we provided you with the list of English stopwords.
4. Optionally, any other preprocessing that you deem necessary.

All the preprocessing code must be contained in the `preprocessing.py` file.  
You can use Hugginface's [datasets library](https://huggingface.co/docs/datasets/) for easy dataset download.

### Part 1.2 Building vocabulary (1 point)
Before we can feed our text to the model it must be vectorized. We use 300 dimensional pretrained [FastText embeddings](https://fasttext.cc/docs/en/english-vectors.html) for mapping words to vectors. To know more general information about embeddings you can refer to [this video](https://www.youtube.com/watch?v=ERibwqs9p38) (even though, we use different types of embeddings - FastText vs Word2Vec described in the video - the general purpose of them is the same).  
In order to apply the embedding, we must first construct the vocabulary for data. Complete the `create_vocab` method of `STSData` class in `sts_data.py` where you concatenate each sentence pair, tokenize it and construct the vocabulary for the whole training data. You should use [torchtext](https://torchtext.readthedocs.io/en/latest/data.html
) for processing the data. For tokenization, you can use any library (or write your own tokenizer), but we recommend you to use tokenizer by [spacy](https://spacy.io/). Use the `fasttext.simple.300d` as pretrained vectors.  
In the end, you must have a vocabulary object capable of mapping your input to corresponding vectors. Remember that the vocabulary is created using only training data (not touching validation or test data).

### Part 1.3 Creating DataLoader (1 point)
Implement `get_data_loader` method of `STSData` class in `sts_data.py`. It must perform the following operations on each of the data splits:
1. vectorize each pair of the sentences by replacing all tokens with their index in vocabulary
2. normalize labels
3. convert everything to PyTorch tensors
4. pad every sentence so that all of them have the same length
5. create `STSDataset` from `dataset.py`
6. create PyTorch DataLoader out of the created dataset. 


We have provided you with the interfaces of possible helper functions, but you can change them as you need.   
In the end, you must have 3 data loaders for each of the splits.

In [10]:
reload(sts_data)
from sts_data import STSData

columns_mapping = {
        "sent1": "sentence_A",
        "sent2": "sentence_B",
        "label": "relatedness_score",
    }
dataset_name = "sick"
sick_data = STSData(
    dataset_name=dataset_name,
    columns_mapping=columns_mapping,
    normalize_labels=True,
    normalization_const=5.0,
)
batch_size = 64
sick_dataloaders = sick_data.get_data_loader(batch_size=batch_size)

INFO:root:loading and preprocessing data...


Downloading:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/218k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

...

Dataset sick downloaded and prepared to /root/.cache/huggingface/datasets/sick/default/0.0.0/c6b3b0b44eb84b134851396d6d464e5cb8f026960519d640e087fe33472626db. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

INFO:root:reading and preprocessing data completed...


train- 4439
                                   sentence_A  \
1  children playing house standing background   
2        boys playing outdoors smiling nearby   
3                 kids playing outdoors smile   
4        boys playing outdoors smiling nearby   
5                               dogs fighting   

                              sentence_B  relatedness_score  
1  kids playing yard standing background                3.2  
2            kids playing outdoors smile                4.7  
3  kids playing yard standing background                3.4  
4  kids playing yard standing background                3.7  
5                 dogs wrestling hugging                4.0  


INFO:root:creating vocabulary...
INFO:torchtext.vocab.vectors:Downloading vectors from https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.simple.vec
.vector_cache/wiki.simple.vec: 293MB [00:13, 22.2MB/s]                           
INFO:torchtext.vocab.vectors:Extracting vectors into .vector_cache
INFO:torchtext.vocab.vectors:Loading vectors from .vector_cache/wiki.simple.vec
100%|██████████| 111051/111051 [00:15<00:00, 7112.06it/s]
INFO:torchtext.vocab.vectors:Saving vectors to .vector_cache/wiki.simple.vec.pt
INFO:root:creating vocabulary completed...


## Part 2. Model Configuration & Hyperparameter Tuning (3 points)
In this part, you are required to define a model capable of learning self-attentive sentence embeddings described in [this ICLR paper](https://arxiv.org/pdf/1703.03130.pdf). The sentence embedding learned by this model will be used for computing the similarity score instead of the simpler embeddings described in the original AAAI paper.  
Please familiarize yourself with the model described in the ICLR paper and implement `SiameseBiLSTMAttention` and `SelfAttention` classes in `siamese_lstm_attention.py`. Remember that you must run the model on each sentence in the sentence pair to calculate the similarity between them. You can use `similarity_score` from `utils.py` to compute the similarity score between two sentences. 
  
To get more theoretical information about attention mechanisms you can refer to [this chapter](https://web.stanford.edu/~jurafsky/slp3/10.pdf) of ["Speech and Language Processing" book](https://web.stanford.edu/~jurafsky/slp3/) by Dan Jurafsky and James H. Martin, where the attention mechanism is described in the context of the machine translation task. 

Finally, once your implementation works on the default parameters stated below, make sure to perform **hyperparameter tuning** to find the best combination of hyperparameters.

In [11]:
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

output_size = 1
hidden_sizes = [32, 64, 128]
vocab_size = len(sick_data.vocab)
embedding_size = 300
embedding_weights = sick_data.vocab.vectors
lstm_layers = [2,4]
learning_rates = [1e-02, 1e-3, 1e-04, 5e-03]
fc_hidden_size = 64
max_epochs = 30
bidirectional = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
## self attention config
self_attention_config = {
    "hidden_size": 150,  ## refers to variable 'da' in the ICLR paper
    "output_size": 20,  ## refers to variable 'r' in the ICLR paper
    "penalty": 1.0,  ## refers to penalty coefficient term in the ICLR paper
    "model_name": "siamese_lstm_attention"
}

In [12]:
## init siamese lstm
from siamese_lstm_attention import SiameseBiLSTMAttention
max_accuracy = 0
for hidden_size in hidden_sizes:
    for lstm_layer in lstm_layers:
        for learning_rate in learning_rates:
            siamese_lstm_attention = SiameseBiLSTMAttention(
                batch_size=batch_size,
                output_size=output_size,
                hidden_size=hidden_size,
                vocab_size=vocab_size,
                embedding_size=embedding_size,
                embedding_weights=embedding_weights,
                lstm_layers=lstm_layer,
                self_attention_config=self_attention_config,
                fc_hidden_size=fc_hidden_size,
                device=device,
                bidirectional=bidirectional,
            )
            siamese_lstm_attention.to(device)
            #hyperparamter training
            optimizer = torch.optim.Adam(list(siamese_lstm_attention.parameters()), lr = learning_rate)
            accuracy = train_model(
                model=siamese_lstm_attention,
                optimizer=optimizer,
                dataloader=sick_dataloaders,
                data=sick_data,
                max_epochs=max_epochs,
                config_dict={
                    "device": device,
                    "model_name": "siamese_lstm_attention",
                    "self_attention_config": self_attention_config,
                },
            )
            if accuracy > max_accuracy:
                max_accuracy = accuracy
                best_learning_rate = learning_rate
                best_hidden_size = hidden_size
                best_lstm_layers = lstm_layer

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch : 1, Iteration : 1 , Loss : 1.125708818435669
Epoch : 1, Iteration : 11 , Loss : 1.14335036277771
Epoch : 1, Iteration : 21 , Loss : 1.1228764057159424
Epoch : 1, Iteration : 31 , Loss : 1.1335511207580566
Epoch : 1, Iteration : 41 , Loss : 1.1427830457687378
Epoch : 1, Iteration : 51 , Loss : 1.1241326332092285
Epoch : 1, Iteration : 61 , Loss : 1.1151238679885864
Epoch : 1 , Loss : 1.1423695951268293


INFO:root:Evaluating accuracy on dev set
INFO:root:new model saved


Epoch : 1, Validation Iteration : 1 , Loss : 1.1252408027648926
Epoch : 1, Validation Loss : 1.1227366753986903


INFO:root:Train loss: 1.1423695951268293 - acc: 0.45177769207614654 -- Validation loss: 1.1227366753986903 - acc: 0.5244010369288449
  3%|▎         | 1/30 [00:02<01:18,  2.70s/it]

...

Epoch : 28 , Loss : 0.5840602879938872
Epoch : 28, Validation Iteration : 1 , Loss : 0.5997210741043091
Epoch : 28, Validation Loss : 0.6019861442702157
Epoch : 29, Iteration : 1 , Loss : 0.5814709663391113
Epoch : 29, Iteration : 11 , Loss : 0.585802435874939
Epoch : 29, Iteration : 21 , Loss : 0.5882184505462646
Epoch : 29, Iteration : 31 , Loss : 0.5888376235961914
Epoch : 29, Iteration : 41 , Loss : 0.5818840265274048
Epoch : 29, Iteration : 51 , Loss : 0.5857512354850769
Epoch : 29, Iteration : 61 , Loss : 0.5865926742553711


INFO:root:Evaluating accuracy on dev set
INFO:root:Train loss: 0.5838573583658191 - acc: 0.7599582823074023 -- Validation loss: 0.6017699837684631 - acc: 0.4873692817472896
 97%|█████████▋| 29/30 [01:27<00:03,  3.02s/it]

Epoch : 29 , Loss : 0.5838573583658191
Epoch : 29, Validation Iteration : 1 , Loss : 0.599687933921814
Epoch : 29, Validation Loss : 0.6017699837684631
Epoch : 30, Iteration : 1 , Loss : 0.5808733105659485
Epoch : 30, Iteration : 11 , Loss : 0.5856252908706665
Epoch : 30, Iteration : 21 , Loss : 0.5912476181983948
Epoch : 30, Iteration : 31 , Loss : 0.5887417793273926
Epoch : 30, Iteration : 41 , Loss : 0.5821812748908997
Epoch : 30, Iteration : 51 , Loss : 0.5849509239196777
Epoch : 30, Iteration : 61 , Loss : 0.5860226154327393


INFO:root:Evaluating accuracy on dev set
INFO:root:Train loss: 0.5839652548665586 - acc: 0.7584295484325707 -- Validation loss: 0.6012201138905117 - acc: 0.4768485081177617
100%|██████████| 30/30 [01:30<00:00,  3.02s/it]

Epoch : 30 , Loss : 0.5839652548665586
Epoch : 30, Validation Iteration : 1 , Loss : 0.5967756509780884
Epoch : 30, Validation Loss : 0.6012201138905117


In [13]:
print('Best Learning Rate : '+str(best_learning_rate))
print('Best LSTM Hidden Size : '+str(best_hidden_size))               
print('Best LSTM Number of Layers : '+str(best_lstm_layers))

Best Learning Rate : 0.0001
Best LSTM Hidden Size : 128
Best LSTM Number of Layers : 2


## Part 3. Training (2 points)  
Perform the final training of the model by implementing functions in `train.py` after setting values of your best-chosen hyperparameters. Note you can use the same training function when performing hyperparameter tuning.
- **What is a good choice of performance metric here for evaluating your model?** [Max 2-3 lines]
  - Since this is a regression problem and similarity measurement experiment between two given sentences a correlation score would be a good choice of performance measure. Here for task1 and task2 we have used spearman correlation as metric two evaluate the models performance on validation set and also on test set.


- **What other performance evaluation metric can we use here for this task? Motivate your answer.**[Max 2-3 lines]

 - We can use other correlation metrics like the spearman such as pearson correlation. Since this is a regression problem we can also use Mean Squared Error on the whole evaluation set as a performance metric.




In [14]:
from datetime import datetime
siamese_lstm_attention = SiameseBiLSTMAttention(
                batch_size=batch_size,
                output_size=output_size,
                hidden_size=best_hidden_size,
                vocab_size=vocab_size,
                embedding_size=embedding_size,
                embedding_weights=embedding_weights,
                lstm_layers=best_lstm_layers,
                self_attention_config=self_attention_config,
                fc_hidden_size=fc_hidden_size,
                device=device,
                bidirectional=bidirectional,
            )
siamese_lstm_attention.to(device)
            #hyperparamter training
optimizer = torch.optim.Adam(list(siamese_lstm_attention.parameters()), lr = best_learning_rate)
start_time = datetime.now()
_ = train_model(
                model=siamese_lstm_attention,
                optimizer=optimizer,
                dataloader=sick_dataloaders,
                data=sick_data,
                max_epochs=max_epochs,
                config_dict={
                    "device": device,
                    "model_name": "siamese_lstm_attention",
                    "self_attention_config": self_attention_config,
                },
            )
end_time = datetime.now()
time_difference = end_time - start_time 
seconds = time_difference.total_seconds()
minutes = seconds / 60
epoch_minutes = minutes / max_epochs

print('Total minutes to train : '+str(minutes))
print('Average minutes per epoch : '+str(epoch_minutes))

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch : 1, Iteration : 1 , Loss : 1.148439645767212
Epoch : 1, Iteration : 11 , Loss : 1.152021884918213
Epoch : 1, Iteration : 21 , Loss : 1.1278976202011108
Epoch : 1, Iteration : 31 , Loss : 1.1345456838607788
Epoch : 1, Iteration : 41 , Loss : 1.140523910522461
Epoch : 1, Iteration : 51 , Loss : 1.128117561340332


INFO:root:Evaluating accuracy on dev set


Epoch : 1, Iteration : 61 , Loss : 1.11466646194458
Epoch : 1 , Loss : 1.1360439217608909


INFO:root:new model saved
INFO:root:Train loss: 1.1360439217608909 - acc: 0.4186417036359317 -- Validation loss: 1.1240346261433192 - acc: 0.5289804048178834
  3%|▎         | 1/30 [00:01<00:52,  1.82s/it]

...

Epoch : 29, Validation Iteration : 1 , Loss : 0.6141866445541382
Epoch : 29, Validation Loss : 0.6134417653083801
Epoch : 30, Iteration : 1 , Loss : 0.5962571501731873
Epoch : 30, Iteration : 11 , Loss : 0.6032271385192871
Epoch : 30, Iteration : 21 , Loss : 0.6019928455352783
Epoch : 30, Iteration : 31 , Loss : 0.605195164680481
Epoch : 30, Iteration : 41 , Loss : 0.5967926979064941
Epoch : 30, Iteration : 51 , Loss : 0.6027363538742065


INFO:root:Evaluating accuracy on dev set


Epoch : 30, Iteration : 61 , Loss : 0.5983301401138306
Epoch : 30 , Loss : 0.6000810619713604


INFO:root:Train loss: 0.6000810619713604 - acc: 0.7941718892473039 -- Validation loss: 0.609944828919002 - acc: 0.5818865167342394
100%|██████████| 30/30 [00:53<00:00,  1.78s/it]

Epoch : 30, Validation Iteration : 1 , Loss : 0.6118080615997314
Epoch : 30, Validation Loss : 0.609944828919002
Total minutes to train : 0.8921885833333333
Average minutes per epoch : 0.02973961944444444


## Part 4. Evaluation and Analysis (2 points)  
Implement function evaluate_test_set to calculate the final accuracy of the performance evaluation metric on the test data.  
Compare the result with the original AAAI paper. Сomment on effect of penalty loss on model capacity. Did the inclusion of the self-attention block improve the results? If yes, then how? Can you think of additional techniques to improve the results? Briefly answer these questions in the markdown cells.

In [15]:
evaluate_test_set(
    model=siamese_lstm_attention,
    dataloader=sick_dataloaders,
    config_dict={
        "device": device,
        "self_attention_config": self_attention_config,
        "model_name": "siamese_lstm_attention.pth"
    },
)

INFO:root:Evaluating accuracy on test set


Spearman Correlation : 0.6020277058525634
Pearson Correlation : 0.6794484079899329
Mean Squared Error : 0.02382063865661621


     
     We are reporting spearman correlation, pearson correlation and MSE loss on the test set in Table 1. From numbers of the table it is clear that self-attention mechanism helps the bilstm model to improve the performance by several percent. Even it reduces the MSE loss on the test set as well. Attentions helps the model to focus on some  particular part of the sentence which will help the model to generate a relatedness score which is similar to the actual relatedness score.Penalty loss is working as a regularizer here. For the second model reported in the table this regularizer has significant positive effect which is backed the scores from the table.  But if we compare the results with the AAAI paper  our correlation scores are much lower than the scores reported in the paper. But surprisingly our MSE loss is much lower than the MSE loss reported in the paper. We can improve the results by adding more data to our training data, in the paper they also did that. Then we should finetune the hyperparameters of the  model on the whole new dataset. Because to train a LSTM parameters, 4439 sentence pairs are not enough and training 100s of epochs will overfit the model so we need to include more data.








